In [1]:
import os
import numpy as np
import pandas as pd
from pandas import read_csv
import geopandas as gpd
import fiona
import shapely
from shapely.geometry import Point, Polygon
import geopandas
import xarray as xr

In [4]:
## export a list of unmasked images that intersect each plot point 
# read points. for each point, read L7 images and create list of images that intersect or contain each point         

def list_images(proj_path, grid_list, inputShape):
    plots_df = pd.DataFrame(columns=['InputFID', 'ProdID', 'Parcela', 'L7count' , 'L7images', 'L8count', 'L8images', 'S2count', 'S2images'])

    for grid in grid_list:    
        L7_img_list = []
        L8_img_list = []
        S2_img_list = []
        imagePath = proj_path + "raster/grids/" + grid + "/brdf/"
        for img in sorted(os.listdir(imagePath)):   
            if img.startswith('LE07') & img.endswith('_T1.nc'):
                L7_img_list.append(os.path.join(imagePath,img))
            if img.startswith('LC08') & img.endswith('_T1.nc'):
                L8_img_list.append(os.path.join(imagePath,img))            
            if img.startswith('L1C') & img.endswith('_MTD.nc'):
                S2_img_list.append(os.path.join(imagePath,img))   
        print('number of L7 images in grid', str(grid), ": " , len(L7_img_list))      
        print('number of L8 images in grid', str(grid), ": " , len(L8_img_list))
        print('number of S2 images in grid', str(grid), ": " , len(S2_img_list))
        
        strip_landsat_path = "netcdf:/raid-cel/sandbox" + proj_path[5:15] + proj_path[15:] + "raster/grids/" + grid + "/gee/"
        strip_sentinel_path = proj_path[:15] + proj_path[15:] + "raster/grids/" + grid + "/brdf/"   
        with fiona.open(inputShape) as shp:
            for feature in shp:
                if feature['properties']['UNQ'] == float(grid):
                    siteID = feature['properties']['Target_FID']
                    prodID = feature['properties']['oldIDPROD']
                    parcela = feature['properties']['Parcela']
                    coords = feature['geometry']['coordinates']
                    print('FID: ', str(int(siteID)), str(coords))
                    
                    L7_point_rast_dict = {}
                    for rast in L7_img_list:
                        with xr.open_dataset(rast) as xrimg:
                            point = xrimg.sel(x=coords[0], y=coords[1], method="nearest")
                            val = point['nir']
                            val_arr = val.data
                            L7_rast_dict={point.filename: val_arr}
                            L7_smaller_things = {key: value for key, value in L7_rast_dict.items() if value != 65535}
                        L7_point_rast_dict.update(L7_smaller_things)
                    new_L7_dict = {key.replace(strip_landsat_path, ""): value for key, value in L7_point_rast_dict.items()}
                    feature['properties']['L7'] = list(new_L7_dict.keys())
                    print(len(new_L7_dict), ' images with data from L7')
                    
                    L8_point_rast_dict = {}
                    for rast in L8_img_list:
                        with xr.open_dataset(rast) as xrimg:
                            point = xrimg.sel(x=coords[0], y=coords[1], method="nearest")
                            val = point['nir']
                            val_arr = val.data
                            L8_rast_dict={point.filename: val_arr}
                            L8_smaller_things = {key: value for key, value in L8_rast_dict.items() if value != 65535}
                        L8_point_rast_dict.update(L8_smaller_things)
                    new_L8_dict = {key.replace(strip_landsat_path, ""): value for key, value in L8_point_rast_dict.items()}
                    feature['properties']['L8'] = list(new_L8_dict.keys())
                    print(len(new_L8_dict), 'images with data from L8')
                    
                    S2_point_rast_dict = {}
                    for rast in S2_img_list:
                        with xr.open_dataset(rast) as xrimg:
                            point = xrimg.sel(x=coords[0], y=coords[1], method="nearest")
                            val = point['nir']
                            val_arr = val.data
                            S2_rast_dict={point.filename: val_arr}
                            S2_smaller_things = {key: value for key, value in S2_rast_dict.items() if value != 65535}
                        S2_point_rast_dict.update(S2_smaller_things)
                    new_S2_dict = {key.replace(strip_sentinel_path, ""): value for key, value in S2_point_rast_dict.items()}
                    feature['properties']['S2'] = list(new_S2_dict.keys())
                    print(len(new_S2_dict), 'images with data from S2')
                    
                    feature_list = [int(siteID), prodID, parcela, len(new_L7_dict), list(new_L7_dict.keys()), len(new_L8_dict), list(new_L8_dict.keys()), len(new_S2_dict), list(new_S2_dict.keys())]
                    plots_df.loc[len(plots_df)] = feature_list
                    
    out_path = inputShape[:-4] + "_image_list.csv"
    return plots_df.to_csv(out_path)
        

In [5]:
utm_proj_path = "/home/sandbox-cel/crs_lc/" 
utm_grids = ['002306'] 
utm_inputShape='/home/sandbox-cel/wsa_lc/WSA_AOI/CRS_Plot_Locations/NI_ES_points_UTM16.shp'

albers_proj_path = "/home/sandbox-cel/wsa_lc/"
alb_grids = ['001892','001972','002056','002225','002387','002550','002047','002127']
alb_inputShape='/home/sandbox-cel/wsa_lc/WSA_AOI/CRS_Plot_Locations/NI_ES_points_AEA.shp'

In [6]:
list_images(utm_proj_path, utm_grids, utm_inputShape)

number of L7 images in grid 002306 :  118
number of L8 images in grid 002306 :  126
number of S2 images in grid 002306 :  608
FID:  49 (571890.884343618, 1466655.4401975938)
39  images with data from L7
32 images with data from L8
123 images with data from S2
FID:  87 (571851.6618522261, 1466644.8671247293)
37  images with data from L7
34 images with data from L8
121 images with data from S2


In [5]:
list_images(albers_proj_path, alb_grids, alb_inputShape)

number of L7 images in grid 001892 :  182
number of L8 images in grid 001892 :  165
number of S2 images in grid 001892 :  943
FID:  12 (782824.5417865542, -2909714.02730662)
104  images with data from L7
117 images with data from L8
378 images with data from S2
FID:  13 (785241.3652106337, -2911362.2271313295)
96  images with data from L7
86 images with data from L8
366 images with data from S2
FID:  14 (793149.0362650802, -2909575.5794712026)
99  images with data from L7
114 images with data from L8
381 images with data from S2
FID:  15 (786918.5648299173, -2903309.9748629313)
109  images with data from L7
104 images with data from L8
383 images with data from S2
FID:  16 (786901.3527812023, -2903297.2762822695)
114  images with data from L7
105 images with data from L8
384 images with data from S2
FID:  18 (782541.405014403, -2912117.655081833)
103  images with data from L7
109 images with data from L8
383 images with data from S2
FID:  32 (782827.676008948, -2909720.8217803705)
104 

80 images with data from L8
116 images with data from S2
FID:  111 (1153188.10406169, -3028584.9369747825)
39  images with data from L7
78 images with data from L8
111 images with data from S2
number of L7 images in grid 002047 :  355
number of L8 images in grid 002047 :  314
number of S2 images in grid 002047 :  657
FID:  4 (692558.332045801, -2951676.846479278)
157  images with data from L7
146 images with data from L8
0 images with data from S2
FID:  5 (692564.2592609581, -2951837.562117256)
161  images with data from L7
156 images with data from L8
0 images with data from S2
FID:  6 (692293.8920355623, -2955295.8573893923)
190  images with data from L7
200 images with data from L8
0 images with data from S2
FID:  7 (692198.9921688722, -2955289.3351090327)
195  images with data from L7
201 images with data from L8
0 images with data from S2
FID:  24 (692551.8537946775, -2951677.2988537103)
157  images with data from L7
146 images with data from L8
0 images with data from S2
FID:  25